<a href="https://colab.research.google.com/github/Ramprashanth17/DataEngineering/blob/main/Databricks_DE/Practice/AdvOps_Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Grouping Data and Types of Joins in Spark

We can group data based on different cols in a DataFrame, and apply a different aggregations such as sum or avg to get a holistic view of data slices.

```
salary_data.groupby('Department')
```

This returns a grouped data object and can be assigned to a separate DF and more ops can be done on this data.

Ex:
```
salary_data.groupby('Department').avg().show()
```



### A complex groupBy Statement

groupBy can be used in complex data ops, such as multiple aggregations within a single groupBy statement.



In [ ]:
from pyspark.sql.functions import col, round
salary_data.groupBy('Department')\
  .sum('Salary')\
  .withColumn('sum(Salary)', round(col('sum(Salary)'), 2)) \
  .withColumnRenamed('sum(Salary)', 'Salary') \
  .orderBy('Department')\
  .show()

## Joining DataFrames in Spark

Join ops are used to combine data from two or more DataFrames based on a common column. It is essentially for merging datasets. aggregations, and relational operations.

**.join()** is the method to join and its parameters are:

- **other** The other dataframe to join with.
- **on**: Cols on which to join the DataFrames
- **how**: Type of join
- **suffixes**: Suffixes to add to cols with the same name in both dataframes

```
DF1.join(DF2, on, how)
```



In [ ]:
salary_data_with_id = [(1, "John", "Field-eng", 3500), \
    (2, "Robert", "Sales", 4000), \
    (3, "Maria", "Finance", 3500), \
    (4, "Michael", "Sales", 3000), \
    (5, "Kelly", "Finance", 3500), \
    (6, "Kate", "Finance", 3000), \
    (7, "Martin", "Finance", 3500), \
    (8, "Kiran", "Sales", 2200), \
  ]


columns= ["ID", "Employee", "Department", "Salary"]
salary_data_with_id = spark.createDataFrame(data = salary_data_with_id, schema = columns)
salary_data_with_id.show()

In [ ]:
employee_data = [(1, "NY", "M"), \
    (2, "NC", "M"), \
    (3, "NY", "F"), \
    (4, "TX", "M"), \
    (5, "NY", "F"), \
    (6, "AZ", "F") \
  ]
columns= ["ID", "State", "Gender"]
employee_data = spark.createDataFrame(data = employee_data, schema = columns)
employee_data.show()